In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
# 데이터 불러오기 
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [3]:
print(train_df.shape)
# (batch,컬럼) 형태의 구조
train_df

(60000, 786)


,index,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,6,0,0,0,0,0,0,0,5,...,0,0,0,30,43,0,0,0,0,0
3,3,0,0,0,0,1,2,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,59996,1,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,59997,8,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,59998,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test_df

,index,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,0,0,0,0,0,0,0,0,...,32,23,14,20,0,0,1,0,0,0
9996,9996,0,0,0,0,0,0,0,0,0,...,0,0,0,2,52,23,28,0,0,0
9997,9997,0,0,0,0,0,0,0,0,0,...,175,172,172,182,199,222,42,0,1,0
9998,9998,0,1,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
#학습에 사용하기 위해서는 float형태로 바뀌어야 한다.
train_data = np.array(train_df.iloc[:,1:], dtype = 'float32') #인덱스를 제외하고 넣기
test_data = np.array(test_df.iloc[:,1:], dtype='float32')

In [6]:
x_train = train_data[:, 1:] # 피처들
y_train = train_data[:,0]  # 라벨

x_test = test_data

In [7]:
#validation set 두기 (정확한 평가를 위해서)
#주로 하이퍼 파라미터 찾기 위해서 validation set을 둔다.
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)

print(x_train.shape, x_validate.shape)


(48000, 784) (12000, 784)


In [8]:
# conda install -c conda-forge scikit-image

from skimage.transform import resize
from skimage import data

In [9]:
# reshape를 통해 다시 이미지로 만들어 주기.

x_train = x_train.reshape(-1,28,28)

x_validate = x_validate.reshape(-1,28,28)

x_test = x_test.reshape(-1,28,28)


# efficient_b5가 이미지의 크기는 32,32. 채널은 3채널로 요구해서 사이즈를 변경해준다.



#code : https://www.kaggle.com/code/byeongheonkang/fashion-mnist-with-pretrained-model-tutorial
def resize_all(x, shape = (32,32)):
    band_shape = x.shape
    x_resize = np.zeros(shape = (band_shape[0],shape[0],shape[1]))
    for i in range(band_shape[0]):
        x_resize[i] = resize(x[i], shape)
    return x_resize


def transform_input_data(x):
    x_eff = np.array(x).reshape(-1,28,28)
    x_eff = resize_all(x_eff, (32,32))
    x_eff = np.repeat(x_eff[:, :, :, np.newaxis], 3, axis=3)
#    x_vgg = preprocess_input(x_vgg)
    return x_eff


x_train = transform_input_data(x_train)
x_validate = transform_input_data(x_validate)
x_test = transform_input_data(x_test)

print(x_train.shape)



(48000, 32, 32, 3)


In [10]:
#모델 구축
#https://keras.io/api/applications/
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB5



eff_model = EfficientNetB5(input_shape=(32, 32, 3),
                              include_top=False,
                              weights="imagenet")

eff_model.summary()

Model: "efficientnetb5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 32, 32, 3)    7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 32, 32, 3)    0           ['normalization[0][0]']          
                                                                                     

In [11]:
#모델 구축
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Activation,Flatten

def effb5_model():
    pretrained_model = eff_model
    model = Sequential()

    # Add the vgg convolutional base model
    model.add(pretrained_model)

    # Add new layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))   
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

nn_model= effb5_model()

In [12]:
#compile이란 우리가 만든 모델을 어떻게 weight들을 최적화 시킬지 정해주는 부분.
from keras.optimizers import Adam
nn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])
#Adam은 가중치를 학습 시켜주는 optimizer중 가장 유명한 것.
#learning rate는 데이터, 전처리 방법에 따라 달라진다. 스스로 조절해가며 찾아야한다.
#metrics는 학습하면서 평가해주는 중간결과를 보여줄 때 사용.

#sparse_categorical_crossentropy는 0~10으로된 수치형 라벨을 10개로 만들어진 백터와 매칭시켜준다.

#categorical_crossentropy를 쓰면,0~10칸으로 된 라벨을 10개 벡터와 매칭.


In [13]:
nn_model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_validate,y_validate),
)

Epoch 1/10
1500/1500 [==============================] - 182s 113ms/step - loss: 0.5659 - accuracy: 0.8163 - val_loss: 0.3059 - val_accuracy: 0.8948
Epoch 2/10
1500/1500 [==============================] - 166s 111ms/step - loss: 0.3634 - accuracy: 0.8838 - val_loss: 0.2825 - val_accuracy: 0.9006
Epoch 3/10
1500/1500 [==============================] - 167s 111ms/step - loss: 0.3078 - accuracy: 0.9016 - val_loss: 0.4866 - val_accuracy: 0.8351
Epoch 4/10
1500/1500 [==============================] - 169s 112ms/step - loss: 0.2912 - accuracy: 0.9065 - val_loss: 0.2969 - val_accuracy: 0.9134
Epoch 5/10
1500/1500 [==============================] - 170s 113ms/step - loss: 0.2425 - accuracy: 0.9215 - val_loss: 0.4503 - val_accuracy: 0.8797
Epoch 6/10
1500/1500 [==============================] - 171s 114ms/step - loss: 0.2415 - accuracy: 0.9226 - val_loss: 0.2309 - val_accuracy: 0.9236
Epoch 7/10
1500/1500 [==============================] - 170s 113ms/step - loss: 0.2440 - accuracy: 0.9200 - val_

In [14]:
y_pred = nn_model.predict(x_test)

313/313 [==============================] - 12s 32ms/step


In [15]:
y_pred.shape
#10칸에 대해, 확률들을 적어 둔 것.

(10000, 10)

In [16]:
y_pred

array([[9.9728739e-01, 1.1910307e-05, 1.6781967e-04, ..., 1.3946733e-08,
        5.0965186e-06, 1.3407333e-07],
       [3.6003291e-07, 9.9998772e-01, 2.9458405e-07, ..., 1.5628336e-10,
        3.0960330e-09, 1.7165286e-10],
       [8.9366287e-03, 9.4730749e-05, 9.7516531e-01, ..., 3.3850778e-05,
        1.7865068e-04, 7.3108019e-08],
       ...,
       [5.7628599e-04, 1.3103823e-06, 5.5111213e-06, ..., 5.3051058e-06,
        9.9936432e-01, 2.1488036e-07],
       [7.9660962e-04, 3.5312034e-06, 8.2506385e-06, ..., 7.6126689e-06,
        9.9909186e-01, 5.5007956e-07],
       [1.5135613e-06, 9.9995482e-01, 1.5393213e-06, ..., 1.2913196e-09,
        1.7558689e-08, 1.9231288e-09]], dtype=float32)

In [17]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8')
submission['label'] = y_pred.argmax(axis=1)
submission.to_csv('fashion_submission.csv', index = False)

In [18]:
submission['label'].max()

9

In [19]:

submission

,index,label
0,0,0
1,1,1
2,2,2
3,3,2
4,4,4
...,...,...
9995,9995,0
9996,9996,6
9997,9997,8
9998,9998,8


점수를 높일 수 있는 시도들
1. EPOCH 변화
2. 레이어 변화
3. learning rate 변화
4. ensemble